# Portfolio Optimizer

The portfolio optimizer brings together the power of the Axioma Portfolio Optimizer with Marquee's risk analytics infrastructure
to make minimizing your portfolio's factor risk possible within the same ecosystem.

The optimizer supports two hedging modes:
- **Unidirectional Hedger**: Generates only short positions to hedge your portfolio (default mode)
- **Bidirectional Hedger**: Generates both long and short positions for more flexible factor risk management

To use the optimizer, you must have a license to the Axioma Portfolio Optimizer. Please reach out to the
[Marquee sales team](mailto:gs-marquee-sales@ny.email.gs.com?Subject=Portfolio Optimizer Trial Request)
to learn more about how to get a license or how to bring an existing license to Marquee.

## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import datetime as dt
import pandas as pd

from IPython.display import display
import warnings
from gs_quant.markets.optimizer import (
    OptimizerUniverse,
    FactorConstraint,
    AssetConstraint,
    SectorConstraint,
    IndustryConstraint,
    OptimizerSettings,
    OptimizerStrategy,
    OptimizerConstraints,
    OptimizerObjective,
    OptimizerType,
    OptimizerObjectiveTerm,
    OptimizerObjectiveParameters,
    ConstraintPriorities,
    PrioritySetting,
    CountryConstraint,
    HedgeTarget,
)
from gs_quant.markets.position_set import Position, PositionSet
from gs_quant.markets.securities import Asset, AssetIdentifier
from gs_quant.models.risk_model import FactorRiskModel
from gs_quant.session import GsSession, Environment
from gs_quant.target.hedge import CorporateActionsTypes
from gs_quant.markets.factor_analytics import FactorAnalytics
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.markets.report import ReturnFormat
from gs_quant.api.gs.hedges import GsHedgeApi


client = None
secret = None

## External users must fill in their client ID and secret below and comment out the line below

# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(Environment.PROD, client_id=client, client_secret=secret)
warnings.filterwarnings("ignore", category=RuntimeWarning)

print('GS Session initialized.')

## Step 2: Define Your Initial Position Set

You have two options for defining the initial holdings to optimize:

**Option A**: Create positions manually using identifiers and weights/quantities
**Option B**: Load positions from an existing Marquee portfolio

Choose the option that best fits your workflow. Both approaches will create a `PositionSet` object that can be used in the optimization.

*GS Quant will resolve all identifiers (Bloomberg IDs, SEDOLs, RICs, etc) historically as of the optimization date.*

### Option A: Create New Position Set

Use the `PositionSet` class to define positions manually with identifiers and weights or quantities.

In [ ]:
# Option A: Create positions manually
position_set = PositionSet(
    date=dt.date(day=23, month=4, year=2024),
    reference_notional=10_000_000,
    positions=[Position(identifier='AAPL UW', weight=0.4), Position(identifier='GS UN', weight=0.6)],
)

position_set.resolve()
print(f'Created position set with {len(position_set.positions)} positions')

### Option B: Load Positions from Marquee Portfolio

Load positions from an existing Marquee portfolio using the portfolio ID. This is useful when you want to optimize an existing portfolio that's already managed in Marquee.

In [ ]:
# Option B: Load from existing Marquee portfolio using positions data endpoint
portfolio_id = 'YOUR PORTFOLIO ID HERE'
optimization_date = dt.date(day=11, month=11, year=2025)

# Load portfolio positions using the positions data endpoint
pm = PortfolioManager(portfolio_id)

# Get positions data for the specific date with all relevant fields
portfolio_report = pm.get_performance_report()

positions = portfolio_report.get_portfolio_constituents(
    start_date=optimization_date,
    end_date=optimization_date,
    fields=['grossWeight', 'quantity'],
    return_format=ReturnFormat.JSON,
)

# Create position set from the filtered positions
position_set = PositionSet(
    date=optimization_date,
    positions=[
        Position(
            asset_id=row['assetId'],
            identifier=row['assetId'],
            quantity=row.get('quantity'),
            weight=row.get('grossWeight'),
        )
        for row in positions
    ],
)
print(f'Created position set with {len(position_set.positions)} positions')

### Step 2.1: Analyze Initial Portfolio Risk Exposures

Analyze the risk exposures of the initial portfolio using the Marquee PreTrade.

In [ ]:
# Set parameters for factor analysis in PreTrade
risk_model_id = 'AXIOMA_AXUS4S'
currency = 'USD'
participation_rate = 0.15

analytics = FactorAnalytics(risk_model_id=risk_model_id, currency=currency, participation_rate=participation_rate)

print('Running risk analysis on initial portfolio...')
try:
    initial_analysis = analytics.get_factor_analysis(position_set)

    print('\nInitial Portfolio Summary:')
    summary_table = analytics.create_exposure_summary_table(initial_analysis)
    display(summary_table)
except Exception as e:
    print('\nERROR: Factor analysis failed\n')
    print(f'{str(e)}')
    raise

### Step 2.2: Visualize Initial Portfolio Style Factor Exposures

View the top contributing style factors in your initial portfolio.

In [ ]:
fig = analytics.create_style_factor_chart(initial_analysis, rows=5, title='Initial Portfolio - Style Factor Exposures')
fig.show()

print('\nStyle factors displayed (e.g., Volatility, Market Sensitivity, Momentum, Size, Value, etc.)')

### Step 2.3: Historical Performance Metrics

View dynamic performance metrics for the initial portfolio over the past year.

In [ ]:
# Create dynamic performance chart with time series data from liquidity endpoint
# Shows cumulative PnL and normalized performance over time
performance_chart = analytics.create_dynamic_performance_chart(
    initial_analysis, title='Initial Portfolio - Historical Performance Metrics'
)
performance_chart.show()

print('\nUse the dropdown menu above to toggle between:')
print('- Cumulative PnL')
print('- Normalized Performance')

## Step 3: Define Your Optimizer Universe

An optimizer universe corresponds to the assets that can be used when constructing an optimization, which can be created
using the `OptimizerUniverse` class:

| Parameter | Description | Type| Default Value|
|-----------------|---------------|-------------|-------------
| `assets`      | Assets to include in the universe. |`List[Asset]`| N/A |
| `explode_composites`     | Explode indices, ETFs, and baskets and include their underliers in the universe. |`boolean`| `True` |
| `exclude_initial_position_set_assets`       | Exclude assets in the initial holdings from the optimization. | `boolean` | `False` |
| `exclude_corporate_actions_types`     | Set of of corporate actions to be excluded in the universe. |`List[CorporateActionsTypes]`| `[]` |
| `exclude_hard_to_borrow_assets`       | Exclude hard to borrow assets from the universe. | `boolean` | `False` |
| `exclude_restricted_assets`       | Exclude assets on restricted trading lists from the universe. | `boolean` | `False` |
| `min_market_cap`       | Lowest market cap allowed for any universe constituent. | `float` | `None` |
| `max_market_cap`       | Highest market cap allowed for any universe constituent. | `float` | `None` |

In [ ]:
asset_list = ['SPY US']
asset_list_resolved = [Asset.get(x, AssetIdentifier.BLOOMBERG_COMPOSITE_ID) for x in asset_list]

universe = OptimizerUniverse(
    assets=asset_list_resolved,
    explode_composites=True,
    exclude_initial_position_set_assets=True,
    exclude_restricted_assets=True,
    exclude_hard_to_borrow_assets=True,
    exclude_corporate_actions_types=[CorporateActionsTypes.Mergers],
)

## Step 4: Define Your Risk Model and Factor Risk Constraints

You can run the optimizer using a factor risk model of your choice, so long as you have a license for it, by leveraging
the `FactorRiskModel` class. For any factor in the risk model, you can set more granular constraints on the optimized
portfolio's exposure to the factor.

In this example, let's use the Axioma AXUS4S model and limit the final exposure to Volatility be $10,000 and
the final exposure of Market Sensitivity to be 5,000.

### Pulling Factor Names from Risk Models

You can run the code below to pull the factor names for your model. 

In [ ]:
risk_model = FactorRiskModel.get('AXIOMA_AXUS4S')

risk_model_factors = risk_model.get_many_factors(dt.date(2025, 4, 9))
risk_model_factors_names = pd.DataFrame([x.name for x in risk_model_factors])
pd.set_option('display.max_rows', 87)
print(risk_model_factors_names)

### Set your Factor Constraints

Note that different models might have different sets of factor names, if the code below doesn't work please confirm that the factor name used exists in the Risk Model as shown above. 

In [ ]:
factor_constraints = [
    FactorConstraint(risk_model.get_factor('Volatility'), 10000),
    FactorConstraint(risk_model.get_factor('Market Sensitivity'), 5000),
]

## Step 5: Define Other Optimization Constraints

Outside factor-specific constraints, it's also possible to limit the holding value of individual assets, assets
belonging to a particular GICS sector, and/or assets in a particular country of domicile in the optimization.

In this example, let's constrain the optimization to have 0-50% Microsoft and limit the optimization's notional
coming from Energy and Health Care assets to each be 0-80%. We also limit exposure from gics industry (one level more granular than sector) to be from [0, 50%]

In [ ]:
asset_constraints = [AssetConstraint(Asset.get('MSFT UW', AssetIdentifier.BLOOMBERG_ID), 0, 50)]

sector_constraints = [SectorConstraint('Energy', 0, 80), SectorConstraint('Health Care', 0, 80)]
industry_constraints = [IndustryConstraint('Energy Equipment & Services', 0, 50)]
country_constraints = [CountryConstraint('United States', 0, 100)]

## Step 6: Configure Your Optimization Settings

All other settings for the optimization can be set via the `OptimizerSettings` class:

### Unidirectional Hedger (Default)
Returns only short positions to hedge the target portfolio. Use `allow_long_short=False` (default).

### Bidirectional Hedger
Returns both long and short positions for more flexible hedging. Use `allow_long_short=True` and specify `gross_notional` and `net_notional`.

| Parameter          | Description | Type| Default Value|
|--------------------|---------------|-------------|-------------
| `notional`         | For unidirectional hedges: max notional of hedge (all short positions). For bidirectional: overridden by `gross_notional` |`float`| `10000000` |
| `allow_long_short` | Enable bidirectional hedge mode with both long and short positions |`boolean`| `False` |
| `gross_notional`   | Total absolute notional (\|long\| + \|short\|). Only for bidirectional mode. |`float`| `None` |
| `net_notional`     | Net notional (long - short). Required for bidirectional mode. Use 0 for market neutral. |`float`| `None` |


### Define Constraint Priorities

Priority of the constraint range from 0-5 (prioritized in that order). 
The optimization will fail if it cannot meet a constraint with 0 priority.  
A constraint with priority of 1-5 can be called a relaxed constraint, which means that the optimization will make its best effort to meet the constraint but will not fail if it cannot. \
A constraint with a lower priority will take precedence over a constraint with a higher priority. 


In [ ]:
constraint_priorities = ConstraintPriorities(
    style_factor_exposures=PrioritySetting.ZERO,
    min_sector_weights=PrioritySetting.ONE,
    max_sector_weights=PrioritySetting.ONE,
    min_industry_weights=PrioritySetting.ONE,
    max_industry_weights=PrioritySetting.ONE,
    min_country_weights=PrioritySetting.ONE,
    max_country_weights=PrioritySetting.ONE,
)

### OPTION A: Unidirectional Hedger Configuration

Let's put above parameters in action. In the cell below we configure a **unidirectional hedger** that:

    * Uses tight style exposure constraints
    * Can relax sector, industry, country weight constraints if needed to find a feasible solution
    * Allows 0-100 names in the hedge
    * Sets weight range for constituents from hedge from [1%, 99%]
    * Limits max daily ADV to less than 15%
    * Uses one-sided hedge (short positions only) as allow_long_short is turned off


In [ ]:
# Configure unidirectional hedger (short positions only)
settings = OptimizerSettings(
    allow_long_short=False,  # unidirectional mode: hedge contains only short positions
    constraint_priorities=constraint_priorities,
    min_names=0,
    max_names=100,
    min_weight_per_constituent=0.01,
    max_weight_per_constituent=0.99,
    max_adv=15,
)

### OPTION B: Bidirectional Hedger Configuration

For a **bidirectional hedge** that can take both long and short positions, configure the settings as follows:

**Key differences:**
- Set `allow_long_short=True` to enable both long and short positions
- Use `gross_notional` to control total hedge size (|long| + |short|)
- Use `net_notional` to control net exposure (long - short)
  - Set `net_notional=0` for market neutral hedge
  - Set `net_notional > 0` for net long bias
  - Set `net_notional < 0` for net short bias


**Examples:**
- For a net long bias example: gross_notional=20M, net_notional=10M (e.g., 15M long + 5M short)
- For a net short bias example: gross_notional=20M, net_notional=-10M (e.g., 5M long + 15M short)

In [ ]:
# Example: Bidirectional Hedger with Market Neutral Configuration
settings = OptimizerSettings(
    allow_long_short=True,  # Enable bidirectional mode
    gross_notional=20_000_000,  # Total |long| + |short| = 20M
    net_notional=0,  # Market neutral: long - short = 0 (e.g., 10M long + 10M short)
    constraint_priorities=constraint_priorities,
    max_names=600,
    max_adv=15,
)

## Step 7: Create and Run a Strategy

It's finally time to take all these parameters and construct an optimizer strategy using the `OptimizerStrategy` class in which all the building blocks configured from previous steps are put together. 

    * initial portfolio
    * style, sector, industry and country constraints
    * constraint priorities
    * optimization settings with number of names, weight limits and liquitdy limits on a one sided hedge
    * optimization objectives to minimize on factor risk only


In [ ]:
constraints = OptimizerConstraints(
    asset_constraints=asset_constraints,
    sector_constraints=sector_constraints,
    factor_constraints=factor_constraints,
    industry_constraints=industry_constraints,
)

# define specific risk penalty to be 0 to optimize on factor risk only
risk_term = OptimizerObjectiveTerm(params={'specific_weight': 0, 'factor_weight': 1})
objective_parameters = OptimizerObjectiveParameters(terms=[risk_term])

strategy = OptimizerStrategy(
    initial_position_set=position_set,
    constraints=constraints,
    settings=settings,
    universe=universe,
    risk_model=risk_model,
    objective=OptimizerObjective.MINIMIZE_FACTOR_RISK,
    objective_parameters=objective_parameters,
)

### Option A: Run the hedge with option to save and share it

In [ ]:
# Run the optimization and capture the request/response for saving later
strategy_request, optimization_response = strategy.run_save_share(
    optimizer_type=OptimizerType.AXIOMA_PORTFOLIO_OPTIMIZER
)

# Get hedge and hedged portfolio
hedge = strategy.get_optimization()  # Returns just the optimization results as a PositionSet object
hedged_portfolio = strategy.get_optimized_position_set()

print(f'\nHedge Portfolio: {len(hedge.positions)} positions')
hedge_df = pd.DataFrame(
    [
        {
            'Asset': p.identifier,
            'Quantity': f'{p.quantity:.0f}' if p.quantity else '',
            'Weight': f'{p.weight:.2%}' if p.weight else '',
        }
        for p in hedge.positions
    ]
)
display(hedge_df)

### Option B: Run the hedge with NO option to save and share it

In [ ]:
# Run the optimization and capture the request/response for saving later
strategy.run(optimizer_type=OptimizerType.AXIOMA_PORTFOLIO_OPTIMIZER)

# Get hedge and hedged portfolio
hedge = strategy.get_optimization()  # Returns just the optimization results as a PositionSet object
hedged_portfolio = strategy.get_optimized_position_set()

print(f'\nHedge Portfolio: {len(hedge.positions)} positions')
hedge_df = pd.DataFrame(
    [
        {
            'Asset': p.identifier,
            'Quantity': f'{p.quantity:.0f}' if p.quantity else '',
            'Weight': f'{p.weight:.2%}' if p.weight else '',
        }
        for p in hedge.positions
    ]
)
display(hedge_df)

### Analyze Hedge Composition

View detailed exposure metrics for the hedge, including gross/net/long/short exposures. This is especially useful for long/short hedges.

In [ ]:
# Get detailed exposure summary
exposure_summary = strategy.get_hedge_exposure_summary()

print('=' * 80)
print('HEDGE EXPOSURE SUMMARY')
print('=' * 80)

# Display hedge information
hedge_info = exposure_summary['hedge']
print(f"\nHedge Mode: {hedge_info.get('mode', 'N/A')}")
print(f"Number of Positions: {hedge_info.get('number_of_positions', 0)}")
print(f"Gross Exposure: ${hedge_info.get('gross_exposure', 0):,.0f}")
print(f"Net Exposure: ${hedge_info.get('net_exposure', 0):,.0f}")
print(f"Long Exposure: ${hedge_info.get('long_exposure', 0):,.0f}")
print(f"Short Exposure: ${hedge_info.get('short_exposure', 0):,.0f}")

# Display target portfolio information
print('\n' + '-' * 80)
print('TARGET PORTFOLIO')
print('-' * 80)
target_info = exposure_summary['target']
print(f"Number of Positions: {target_info.get('number_of_positions', 0)}")
print(f"Gross Exposure: ${target_info.get('gross_exposure', 0):,.0f}")
print(f"Net Exposure: ${target_info.get('net_exposure', 0):,.0f}")

# Display hedged portfolio information
print('\n' + '-' * 80)
print('HEDGED TARGET PORTFOLIO (Target + Hedge)')
print('-' * 80)
hedged_info = exposure_summary['hedged_target']
print(f"Number of Positions: {hedged_info.get('number_of_positions', 0)}")
print(f"Gross Exposure: ${hedged_info.get('gross_exposure', 0):,.0f}")
print(f"Net Exposure: ${hedged_info.get('net_exposure', 0):,.0f}")
print(f"Long Exposure: ${hedged_info.get('long_exposure', 0):,.0f}")
print(f"Short Exposure: ${hedged_info.get('short_exposure', 0):,.0f}")
print('=' * 80)

### Split Hedge Constituents by Direction

For bidirectional hedges, view the long and short positions separately to understand each side of the hedge.

In [ ]:
# Split hedge into long and short positions
constituents_by_direction = strategy.get_hedge_constituents_by_direction()

summary = constituents_by_direction['summary']
long_positions = constituents_by_direction['long_positions']
short_positions = constituents_by_direction['short_positions']

print("\nHedge Breakdown:")
print(f"  Long Positions: {summary['num_long']} positions, Total Notional: ${summary['total_long_notional']:,.0f}")
print(f"  Short Positions: {summary['num_short']} positions, Total Notional: ${summary['total_short_notional']:,.0f}")

if len(long_positions) > 0:
    print("\nTop 10 Long Positions:")
    display(long_positions[['assetId', 'name', 'notional']].head(10))
else:
    print("\nNo long positions in hedge (Unidirectional mode: short positions only)")

if len(short_positions) > 0:
    print("\nTop 10 Short Positions:")
    display(short_positions[['assetId', 'name', 'notional']].head(10))
else:
    print("\nNo short positions in hedge")

### Step 7.1: Analyze Hedged Portfolio Performance

View the cumulative PnL performance of the hedged portfolio (target + hedge combined).

In [ ]:
performance_df = strategy.get_cumulative_pnl_performance(HedgeTarget.HEDGED_TARGET)

if not performance_df.empty:
    hedge_mode = 'Bidirectional' if settings.allow_long_short else 'Unidirectional (Short Positions Only)'
    print(f'\nHedge Mode: {hedge_mode}')
    print('Hedged Portfolio Cumulative PnL Performance:')
    fig = analytics.create_performance_chart(
        performance_df, metric='cumulativePnl', title=f'Hedged Portfolio - Cumulative PnL ({hedge_mode})'
    )
    fig.show()
else:
    print('No performance data available')

### Step 7.2: Visualize Hedged Portfolio Style Factor Exposures

View the style factor exposures after hedging.

In [ ]:
# Create style factor chart (excludes GICS classification data)
hedged_factor_exposures = strategy.get_style_factor_exposures(HedgeTarget.HEDGED_TARGET)
hedged_analysis = analytics.convert_hedge_factor_exposures(hedged_factor_exposures)

hedge_mode = 'Bidirectional' if settings.allow_long_short else 'Unidirectional'
fig = analytics.create_style_factor_chart(
    hedged_analysis, rows=5, title=f'Hedged Portfolio - Style Factor Exposures ({hedge_mode} Hedge)'
)
fig.show()

print('\nStyle factors displayed (e.g., Volatility, Market Sensitivity, Momentum, Size, Value, etc.)')
print(f'Hedge Mode: {hedge_mode}')

### Step 7.3: Compare Initial vs Hedged Portfolio

Compare key risk metrics between the initial and hedged portfolios.

In [ ]:
# Display performance summary from strategy in separate tables
hedge_mode = 'Bidirectional Hedger' if settings.allow_long_short else 'Unidirectional Hedger (Short Positions Only)'
print(f'\nPerformance Summary from Hedge Calculation ({hedge_mode}):')
try:
    summary = strategy.get_performance_summary()

    # Optionally display all metrics in one combined table
    print('\n--- All Metrics (Combined) ---')
    display(summary['combined'])

    if settings.allow_long_short:
        print('\nNote: Bidirectional hedge includes both long and short positions for more flexible factor management.')
    else:
        print('\nNote: Unidirectional hedge contains only short positions to offset the target portfolio.')

except Exception as e:
    print(f'Could not retrieve performance summary: {str(e)}')

#### Side-by-Side Factor Comparison

Compare all style factor exposures between initial and hedged portfolios using a grouped bar chart.

In [ ]:
# Create side-by-side comparison of all style factors
hedge_mode = 'Bidirectional' if settings.allow_long_short else 'Unidirectional'
print(f'Creating Style Factor Comparison Chart ({hedge_mode} Hedge)...\n')
comparison_chart = analytics.create_factor_heatmap_comparison(
    initial_analysis, hedged_analysis, title=f'Style Factor Comparison: Initial vs Hedged Target ({hedge_mode})'
)
comparison_chart.show()

print('\nThis chart displays all style factors with side-by-side bars for easy comparison.')
print('Blue bars = Initial Portfolio | Green bars = Hedged Target Portfolio')
print('Factors are sorted by initial portfolio exposure (largest to smallest by absolute value).')
print('A vertical line at zero helps identify positive vs negative exposures.')
print(f'\nHedge Mode: {hedge_mode}')
if settings.allow_long_short:
    print('  - Bidirectional hedge can take both long and short positions')
else:
    print('  - Unidirectional hedge contains only short positions')

#### Note:
If code above fails to find optimization, relax the constraints and try again.

## Step 9: Save Your Hedge to Marquee

Once you're satisfied with your hedge results, you can save them to Marquee for future reference and analysis. This will create a hedge group that can be accessed through the Marquee UI.

In [ ]:
# Save the hedge using the strategy's built-in save_to_marquee method
# This method handles building the payload and posting to the API

saved_hedge_response = strategy.save_to_marquee(
    strategy_request=strategy_request,
    optimization_response=optimization_response,
    hedge_name="Factor Hedge",
    group_name="My Factor Hedge Group",
)

### Step 9.1: Share Your Hedge with Other Users (Optional)

After saving your hedge, you can share it with other users or groups by updating the entitlements. 

In [ ]:
# For view-only access:
hedge_view_emails = ["YOUR EMAIL"]

# Uncomment the following lines to share the hedge:
if saved_hedge_response and saved_hedge_response.get('id'):
    hedge_group_id = saved_hedge_response['id']
    share_response = GsHedgeApi.share_hedge_group(
        hedge_group_id=hedge_group_id,
        view_emails=hedge_view_emails,
        strategy_request=strategy_request,
        optimization_response=optimization_response,
        admin_emails=[],  # Add admin emails here if needed
    )

## Step 10: Create a Basket or Portfolio With Your Results

Now that you have saved your hedge, you can also create a basket or portfolio with your optimization results by following these tutorials:

- [Create a Basket](https://nbviewer.org/github/goldmansachs/gs-quant/blob/master/gs_quant/documentation/06_baskets/tutorials/Basket%20Create.ipynb)
- [Create a Portfolio](https://nbviewer.org/github/goldmansachs/gs-quant/blob/master/gs_quant/documentation/10_one_delta/scripts/portfolios/01_Create%20New%20Historical%20Portfolio.ipynb)

*Other questions? Reach out to the [Portfolio Analytics team](mailto:gs-marquee-analytics-support@gs.com)!*
